In [1]:
from scipy.io import loadmat
from mvlearn.embed import MCCA, KMCCA
import numpy as np
from mvlearn.compose import ViewTransformer, ConcatMerger
from imblearn.over_sampling import SMOTE
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing

import warnings
warnings.filterwarnings("ignore")

In [2]:
mcca = MCCA(n_components=15)
merge = ConcatMerger()

kfold = StratifiedKFold(n_splits=5, shuffle=True)
Sensitive = []
Specificity = []
Precision = []

In [3]:
data_path = r"D:\WorkSpace\DataSets\To石延新-苹果检测光电数据\何进荣处理后的数据\HSI_Dielectric_Data.mat"
data = loadmat(data_path)

y = data['Label']
Xs0 = data["Data1"].T
Xs1 = data["Data2"].T

In [4]:
Xs = [Xs0, Xs1]

X_transformed = merge.fit_transform(Xs)
print(X_transformed.shape)

X_resampled_smote, y_resampled_smote = SMOTE().fit_sample(X_transformed, y)

(265, 399)


In [5]:
for i in range(20):
    for train, test in kfold.split(X_resampled_smote, y_resampled_smote):
        
        stand_means1 = preprocessing.MinMaxScaler()
        stand_means2 = preprocessing.MinMaxScaler()
        
        # 训练集
        X_train = X_resampled_smote[train]
        Xs_0_train = stand_means1.fit_transform(X_train[:,0:256])
        Xs_1_train = stand_means2.fit_transform(X_train[:,256:])
        X_train_list = [Xs_0_train, Xs_1_train]
        X_train_mcca = mcca.fit_transform(X_train_list)
        X_train_transformed = merge.fit_transform(X_train_mcca)
        
        clf = SVC(kernel="poly", degree = 3)
        #clf = DecisionTreeClassifier(max_depth=30)
        clf.fit(X_train_transformed, y_resampled_smote[train])
        
        # 测试集
        X_test = X_resampled_smote[test]
        Xs_0_test = stand_means1.transform(X_test[:,0:256])
        Xs_1_test = stand_means2.transform(X_test[:,256:])
        X_test_list = [Xs_0_test, Xs_1_test]
        X_test_mcca = mcca.transform(X_test_list)
        X_test_transformed = merge.fit_transform(X_test_mcca)
        
        
        predictions = clf.predict(X_test_transformed)
        precisions, recall, f1score, _ = metrics.precision_recall_fscore_support(y_resampled_smote[test], predictions.round())

        Sensitive.append(precisions[0])
        Specificity.append(precisions[1])
        Precision.append(recall[0])

ValueError: Eigenvalue problem has a singular matrix. Add regularization (set `regs` to nonzero value) or reduce the rank (set `signal_rank` low enough).

In [6]:
print("灵敏度：{:.4f}±{:.4f}".format(np.mean(Sensitive), np.std(Sensitive)))
print("特异度：{:.4f}±{:.4f}".format(np.mean(Specificity), np.std(Specificity)))
print("精度：{:.4f}±{:.4f}".format(np.mean(Precision), np.std(Precision)))
print("灵敏度_标准差: %.4f"%np.std(Sensitive))

灵敏度：0.7246±0.0669
特异度：0.7140±0.0494
精度：0.7069±0.0826
灵敏度_标准差: 0.0669
